In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
# import other functions
from imputer import *
from feature_eng import *
from drop import *

In [2]:
df = pd.read_csv("../data/train_data.zip")
df.head()

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1804425,8,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,90.0,244.20,0.157475,0.009783,0.147692
1,1812706,2,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,27.0,258.95,0.157475,0.009783,0.147692
2,1812706,3,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,27.0,258.95,0.157475,0.009783,0.147692
3,1812706,11,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,24.0,258.95,0.157475,0.009783,0.147692
4,1812706,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,258.95,0.157475,0.009783,0.147692


In [4]:
# drop rows missing target variable
df = drop_missing_unacast(df)

# create X and y
X = df.drop('unacast_session_count', axis=1)
y = df.loc[:, 'unacast_session_count']
# split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    test_size=0.2,
                                                      random_state=2020)
# impute NaN values
result = impute_data(X_train, X_valid)
X_train = result[0]
X_valid = result[1] 
# perform feature eng
X_train = comb_cols(X_train)
X_valid = comb_cols(X_valid)
# perform dropping
X_train = drop_columns(X_train)
X_valid = drop_columns(X_valid)

# # perform OHE (climate, density_class, income_class)
# X_train_valid = clean_categorical(X_train, X_valid)
# X_train = X_train_valid[0]
# X_valid = X_train_valid[1]

In [32]:
train_processed = pd.concat([X_train, y_train.reset_index(drop = True)], axis = 1)
valid_processed = pd.concat([X_valid, y_valid.reset_index(drop = True)], axis = 1)

train_processed.to_csv('../data/train_processed.csv', index = False)
valid_processed.to_csv('../data/valid_processed.csv', index = False)

In [31]:
train_processed.head()

,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,monthly_count_ramp,...,monthly_ws_light_br,monthly_ws_gentle_br,monthly_ws_moderate_br,historic_ws_calm,historic_ws_light_air,historic_ws_light_br,historic_ws_gentle_br,historic_ws_moderate_br,avg_fertility_rate,unacast_session_count
0,5,2018,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,63.906875,190.0
1,4,2019,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0,...,0,0,0,1.0,18.0,63.0,41.0,54.0,61.173125,7.0
2,6,2018,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0,...,0,0,0,0.0,2.0,2.0,2.0,3.0,65.725000,32.0
3,9,2019,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0,...,0,0,0,0.0,6.0,7.0,4.0,5.0,65.127500,2332.0
4,7,2019,176,16,160,1.540398e+06,438295.454545,156642.045455,56107.954545,0,...,54,58,35,0.0,13.0,50.0,23.0,20.0,58.470000,156.0


In [33]:
y_train.head()

28601     190.0
27003       7.0
36815      32.0
27301    2332.0
1400      156.0
Name: unacast_session_count, dtype: float64

In [28]:
X_train.index.max()

39591